#### Analyzing Farmburg's A/B Test
Brian is a Product Manager at FarmBurg, a company that makes a farming simulation social network game. In the FarmBurg game, you can plow, plant, and harvest different crops. ​Brian has been conducting an A/B Test with three different variants, and he wants you to help him analyze the results.

Brian ran an A/B test with three different groups: A, B, and C. He has provided us with a CSV file of his results named clicks.csv. It has the following columns:

- user_id: a unique id for each visitor to the FarmBurg site
- group: either 'A', 'B', or 'C' depending on which group the visitor was assigned to
- is_purchase: either 'Yes' if the visitor made a purchase or 'No' if they did not.

In [1]:
import pandas as pd
import numpy as np

In [2]:
abdata = pd.read_csv('clicks.csv')
abdata.head()

,user_id,group,is_purchase
0,8e27bf9a,A,No
1,eb89e6f0,A,No
2,7119106a,A,No
3,e53781ff,A,No
4,02d48cf1,A,Yes


In [3]:
unique_groups = abdata['group'].unique()
unique_groups

array(['A', 'B', 'C'], dtype=object)

In [4]:
abdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4998 entries, 0 to 4997
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      4998 non-null   object
 1   group        4998 non-null   object
 2   is_purchase  4998 non-null   object
dtypes: object(3)
memory usage: 117.3+ KB


In [5]:
abdata.describe()

,user_id,group,is_purchase
count,4998,4998,4998
unique,4998,3,2
top,8e27bf9a,A,No
freq,1,1666,4416


Note that we have two categorical variables: group and is_purchase. We are interested in whether visitors are more likely to make a purchase if they are in any one group compared to the others. Because we want to know if there is an association between two categorical variables, we’ll start by using a Chi-Square test to address our question.

In [6]:
#Making the cross-tabulation with a total column
Xtab = pd.crosstab(abdata['group'], abdata['is_purchase'], margins = True, margins_name = 'Total')
print(Xtab)

is_purchase    No  Yes  Total
group                        
A            1350  316   1666
B            1483  183   1666
C            1583   83   1666
Total        4416  582   4998


In [7]:
Xtab_use_this = pd.crosstab(abdata['group'], abdata['is_purchase'])
print(Xtab)

is_purchase    No  Yes  Total
group                        
A            1350  316   1666
B            1483  183   1666
C            1583   83   1666
Total        4416  582   4998


In [8]:
#Importing chi^2 for the test
from scipy.stats import chi2_contingency

#Performing chi^2 test using a significant threshold of 0.05
chi2, pval, dof, expected = chi2_contingency(Xtab_use_this)
print(pval)

2.4126213546684264e-35


The p-value is approximately 0.0000000000000000000000000000000000241. Therefore, the p-value is less than 0.05 and we can conclude that there is a significant difference in the purchase rate for groups A, B, and C.

Our day is a little less busy than expected, so we decide to ask Brian about his test.

Us: Hey Brian! What was that test you were running anyway?

Brian: We are trying to get users to purchase a small FarmBurg upgrade package. It’s called a microtransaction. We’re not sure how much to charge for it, so we tested three different price points: `$0.99` (group 'A'), `$1.99` (group 'B'), and `$4.99` (group 'C'). It looks like significantly more people bought the upgrade package for `$0.99`, so I guess that’s what we’ll charge.

Us: Oh no! We should have asked you this before we did that Chi-Square test. That wasn’t the right test at all. It’s true that more people wanted to purchase the upgrade at $0.99; you probably expected that. What we really want to know is whether each price point allows us to make enough money that we can exceed some target goal. Brian, how much do you think it cost to build this feature?

Brian: Hmm. I guess that we need to generate a minimum of $1000 in revenue per week in order to justify this project.

Us: We have some work to do!

In order to justify this feature, we will need to calculate the necessary purchase rate for each price point. Let’s start by calculating the number of visitors to the site this week.

It turns out that Brian ran his original test over the course of a week, so the number of visitors in abdata is equal to the number of visitors in a typical week.

Now that we know how many visitors we generally get each week (num_visits), we need to calculate the number of visitors who would need to purchase the upgrade package at each price point `($0.99, $1.99, $4.99)` in order to generate Brian’s minimum revenue target of `$1,000` per week.

In [9]:
#Number of visits per week
num_visits = len(abdata)

#Number of sales needed to reach 1000 dollars at a price point of 0.99
num_sales_needed_099 = 1000/0.99

print('Number of visits per week:', num_visits, '\n')
print('Number of sales needed to reach 1000 dollars at 0.99:', num_sales_needed_099)

Number of visits per week: 4998 

Number of sales needed to reach 1000 dollars at 0.99: 1010.1010101010102


Now that we know how many sales we need at a `$0.99` price point, we have to calculate the proportion of weekly visitors who would need to make a purchase in order to meet that goal.

In [10]:
#proportion of visits needed to reach number of sales at 0.99
p_sales_needed_099 = num_sales_needed_099/num_visits
print(p_sales_needed_099)

0.20210104243717691


The steps will be repeated for the other price points `($1.99 and $4.99)`.

In [11]:
#Number of sales needed to reach 1000 dollars at a price point of 1.99
num_sales_needed_199 = 1000/1.99


#proportion of visits needed to reach number of sales at 1.99
p_sales_needed_199 = num_sales_needed_199/num_visits

print('Number of sales needed to reach 1000 dollars at 1.99:', num_sales_needed_199)
print('Proportion of visits needed to reach the number of sales at 1.99:', p_sales_needed_199)

Number of sales needed to reach 1000 dollars at 1.99: 502.51256281407035
Proportion of visits needed to reach the number of sales at 1.99: 0.10054272965467594


In [12]:
#Number of sales needed to reach 1000 dollars at a price point of 4.99
num_sales_needed_499 = 1000/4.99


#proportion of visits needed to reach number of sales at 4.99
p_sales_needed_499 = num_sales_needed_499/num_visits

print('Number of sales needed to reach 1000 dollars at 4.99:',num_sales_needed_499)
print('Proportion of visits needed to reach the number of sales at 4.99:',p_sales_needed_499)

Number of sales needed to reach 1000 dollars at 4.99: 200.40080160320642
Proportion of visits needed to reach the number of sales at 4.99: 0.040096198800161346


Now let’s return to Brian’s question. To start, we want to know if the percent of Group A `(the $0.99 price point)` that purchased an upgrade package is significantly greater than p_sales_needed_099 (the percent of visitors who need to buy an upgrade package at `$0.99` in order to make our minimum revenue target of `$1,000`).

To answer this question, we want to focus on just the visitors in group A. Then, we want to compare the number of purchases in that group to p_sales_needed_099.

Since we have a single sample of categorical data and want to compare it to a hypothetical population value, a binomial test is appropriate. In order to run a binomial test for group A, we need to know two pieces of information:

The number of visitors in group A (the number of visitors who were offered the `$0.99` price point)
The number of visitors in Group A who made a purchase


In [14]:
#Sample Size of group A
samp_size_099 = np.sum(abdata.group == 'A')

#Number of group A members who made a purchase
sales_099 = np.sum((abdata.group == 'A') & (abdata.is_purchase == 'Yes'))

print('Number of people offered the price 0.99:', samp_size_099)
print('Number who purchased at 0.99:', sales_099)

Number of people offered the price 0.99: 1666
Number who purchased at 0.99: 316


In [15]:
#Sample Size of group B
samp_size_199 = np.sum(abdata.group == 'B')

#Number of group A members who made a purchase
sales_199 = np.sum((abdata.group == 'B') & (abdata.is_purchase == 'Yes'))

print('Number of people offered the price 1.99:', samp_size_199)
print('Number who purchased at 1.99:', sales_199)

Number of people offered the price 1.99: 1666
Number who purchased at 1.99: 183


In [16]:
#Sample Size of group C
samp_size_499 = np.sum(abdata.group == 'C')

#Number of group A members who made a purchase
sales_499 = np.sum((abdata.group == 'C') & (abdata.is_purchase == 'Yes'))

print('Number of people offered the price 4.99:', samp_size_499)
print('Number who purchased at 4.99:', sales_499)

Number of people offered the price 4.99: 1666
Number who purchased at 4.99: 83


For Group A `($0.99 price point)`, we will perform a binomial test using binom_test() to see if the observed purchase rate is significantly greater than number of sales needed to reach 1000 dollars (`p_sales_needed_099`). The same tests will be conducted for the other groups as well.

In [18]:
from scipy.stats import binom_test

In [20]:
#Binom Test for A
pval = binom_test(sales_099, samp_size_099, p_sales_needed_099, alternative = 'greater')
print('P-Value group A:', pval)

P-Value group A: 0.9028081076188554


In [21]:
#Binom Test for B
pval = binom_test(sales_199, samp_size_199, p_sales_needed_199, alternative = 'greater')
print('P-Value group B:', pval)

P-Value group B: 0.11184562623740596


In [22]:
#Binom Test for B
pval = binom_test(sales_499, samp_size_499, p_sales_needed_499, alternative = 'greater')
print('P-Value group C:', pval)

P-Value group C: 0.02794482665983064


`alternative` will indicate the alternative hypothesis for this test; in this case, we want to know if the observed purchase rate is significantly 'greater' than the purchase rate that results in the minimum revenue target.

`P-Value group C` is the only p-value below the threshold of 0.05. Therefore, the C group is the only group where we would conclude that the purchase rate is significantly higher than the target needed to reach `$1000` revenue per week. Therefore, Brian should charge `$4.99` for the upgrade.